In [26]:
import requests
import os
import datetime
import json
from mimetypes import MimeTypes

In [27]:
import pandas as pd

In [28]:
# Fishingbooker credentials (account ID 3694081)
credentials = {
    "client_id": os.environ["BASECAMP_CLIENT_ID"],
    "client_secret": os.environ["BASECAMP_CLIENT_SECRET"],
    "redirect_uri": os.environ["BASECAMP_REDIRECT_URL"],
    "refresh_token": os.environ["BASECAMP_REFRESH_TOKEN"]
}

In [29]:
# Private credentials (account ID 5525374)
credentials_priv = {
    "client_id": "ddc691c9f00f8b7884831b96e46773de9bc04b96",
    "client_secret": "662f2d07a9259780144735ea390c74532470923a",
    "redirect_uri": "https://www.google.com/",
    "refresh_token": "BAhbB0kiAbB7ImNsaWVudF9pZCI6ImRkYzY5MWM5ZjAwZjhiNzg4NDgzMWI5NmU0Njc3M2RlOWJjMDRiOTYiLCJleHBpcmVzX2F0IjoiMjAzMi0xMi0yMVQxMDowODo0M1oiLCJ1c2VyX2lkcyI6WzQ3NDEzNTYxXSwidmVyc2lvbiI6MSwiYXBpX2RlYWRib2x0IjoiMjAxNzg1ZGI0OTNmMzA1MzBmYzQ1YThkNTJmM2QwZDQifQY6BkVUSXU6CVRpbWUNqi4hwGs/vCIJOg1uYW5vX251bWkCOgI6DW5hbm9fZGVuaQY6DXN1Ym1pY3JvIgZXOgl6b25lSSIIVVRDBjsARg==--51c71af8575cdb5870a74d71bfdab636c62974a3"
}

In [30]:
class Basecamp:
    
    def __init__(self, account_id, credentials):
        self.account_id = account_id
        self.credentials = credentials
        self.client_id = credentials["client_id"]
        self.client_secret = credentials["client_secret"]
        self.redirect_uri = credentials["redirect_uri"]
        self.refresh_token = credentials["refresh_token"]
        self.basecamp_account_id = account_id
        self.access_url = f"https://launchpad.37signals.com/authorization/token?type=refresh&refresh_token={self.refresh_token}&client_id={self.client_id}&redirect_uri={self.redirect_uri}&client_secret={self.client_secret}"
        self.access_token = requests.post(self.access_url).json()['access_token']
        self.base_url = f"https://3.basecampapi.com/{self.basecamp_account_id}"
        self.files = []
        
    def upload_file(self, path):
        attachments_url = f"{self.base_url}/attachments.json?name={path}"
        file_size = os.path.getsize(path)
        mime = MimeTypes().guess_type(path)[0]
        headers = {
            'Authorization': 'Bearer '+ self.access_token,
            "Content-Type": mime,
            "Content-Length": str(file_size)
            }

        with open(path, "rb") as content:
            sgid = requests.post(attachments_url, headers=headers, data=content).json()['attachable_sgid']
        file = {
            "filename": os.path.basename(path),
            "file_size": str(file_size),
            "content-type": mime,
            "sgid": sgid
        }
        self.files.append(file)

In [31]:
bc = Basecamp(account_id=5525374, credentials=credentials_priv)

In [32]:
class Campfire:
        
    def __init__(self, project_id, campfire_id, session):
        self.project_id = project_id
        self.campfire_id = campfire_id
        self.files = session.files
        self.access_token = session.access_token
        self.base_url = session.base_url
        self.headers = {
            'Authorization': 'Bearer '+ self.access_token,
            "Content-Type": "application/json"
        }        

    def info(self):
        get_campfire_url = f"{self.base_url}/buckets/{self.project_id}/chats/{self.campfire_id}.json"
        return requests.get(get_campfire_url, headers=self.headers).json()
    
    def get_lines(self):
        get_lines_url = f"{self.base_url}/buckets/{self.project_id}/chats/{self.campfire_id}/lines.json"
        return requests.get(get_lines_url, headers=self.headers).json()
    
    def write(self, content):
        write_url = f"{self.base_url}/buckets/{self.project_id}/chats/{self.campfire_id}/lines.json"
        requests.post(write_url, headers=self.headers, data='{"content": "'+content+'"}')
        print("Sent to campfire successfully!")

In [58]:
#Testing data
project_id = 30698115
campfire_id = 5662447716
message_board_id = 5662447711
message_id = 5662455037

headers = {
    'Authorization': 'Bearer '+ bc.access_token,
    "Content-Type": "application/json"
}

In [59]:
get_message_url = f"{bc.base_url}/buckets/{project_id}/messages/{message_id}.json"
get_message_url

'https://3.basecampapi.com/5525374/buckets/30698115/messages/5662455037.json'

In [60]:
requests.get(get_message_url, headers=headers).json()

{'id': 5662455037,
 'status': 'active',
 'visible_to_clients': False,
 'created_at': '2022-12-21T09:52:45.926Z',
 'updated_at': '2022-12-21T09:53:39.044Z',
 'title': 'test',
 'inherits_status': True,
 'type': 'Message',
 'url': 'https://3.basecampapi.com/5525374/buckets/30698115/messages/5662455037.json',
 'app_url': 'https://3.basecamp.com/5525374/buckets/30698115/messages/5662455037',
 'bookmark_url': 'https://3.basecamp.com/5525374/my/bookmarks/BAh7CEkiCGdpZAY6BkVUSSIuZ2lkOi8vYmMzL1JlY29yZGluZy81NjYyNDU1MDM3P2V4cGlyZXNfaW4GOwBUSSIMcHVycG9zZQY7AFRJIg1yZWFkYWJsZQY7AFRJIg9leHBpcmVzX2F0BjsAVDA=--87d169e0153a6d0f85ee5d1ad0cda043acd0cd05',
 'subscription_url': 'https://3.basecamp.com/5525374/buckets/30698115/recordings/5662455037/subscription',
 'comments_count': 1,
 'comments_url': 'https://3.basecampapi.com/5525374/buckets/30698115/recordings/5662455037/comments.json',
 'parent': {'id': 5662447711,
  'title': 'Message Board',
  'type': 'Message::Board',
  'url': 'https://3.basecampapi.c

In [98]:
class MessageBoard:
    
    def __init__(self, project_id, message_board_id, session):
        self.project_id = project_id
        self.message_board_id = message_board_id
        self.files = session.files
        self.access_token = session.access_token
        self.base_url = session.base_url
        self.headers = {
            'Authorization': 'Bearer '+ self.access_token,
            "Content-Type": "application/json"
        }
    
    def get_all_messages(self):
        get_all_messages_url = f"{self.base_url}/buckets/{self.project_id}/message_boards/{self.message_board_id}/messages.json"
        return requests.get(get_all_messages_url, headers=self.headers).json()
    
    def get_message(self, message_id):
        self.message_id = message_id
        get_message_url = f"{self.base_url}/buckets/{self.project_id}/messages/{self.message_id}.json"
        return requests.get(get_message_url, headers=self.headers).json()
    
    def create_message(self, subject, content):
        self.subject = subject
        self.content = content
        create_message_url = f"{self.base_url}/buckets/{self.project_id}/message_boards/{self.message_board_id}/messages.json"
        requests.post(create_message_url, headers=self.headers, data=str('{"subject": "'+self.subject+'", "content": "'+self.content+'", "status": "active"}').encode())
        print("Message created successfully!")
    
    def update_message(self, message_id, subject, content):
        update_message_url = f"{self.base_url}/buckets/{self.project_id}/messages/{message_id}.json"
        requests.put(update_message_url, headers=self.headers, data=str('{"subject": "'+subject+'", "content": "'+content+'"}').encode())
        print("Message updated successfully!")
        
    def get_all_comments(self, message_id):
        get_all_comments_url = f"{self.base_url}/buckets/{self.project_id}/recordings/{message_id}/comments.json"
        return requests.get(get_all_comments_url, headers=self.headers).json()
    
    def get_comment(self, comment_id):
        self.comment_id = comment_id
        get_comment_url = f"{self.base_url}/buckets/{self.project_id}/comments/{self.comment_id}.json"
        return requests.get(get_comment_url, headers=self.headers).json()
    
    def create_comment(self, message_id, content):
        create_comment_url = f"{self.base_url}/buckets/{self.project_id}/recordings/{message_id}/comments.json"
        requests.post(create_comment_url, headers=self.headers, data='{"content": "'+content+'"}')
        print("Comment created successfully!")
    
    def update_comment(self, comment_id, content):
        update_comment_url = f"{self.base_url}/buckets/{self.project_id}/comments/{comment_id}.json"
        requests.put(update_comment_url, headers=self.headers, data='{"content": "'+content+'"}')
        print("Comment updated successfully!")

In [99]:
mb = MessageBoard(project_id=project_id, message_board_id=message_board_id, session=bc)

In [108]:
mb.update_comment(comment_id=5663320226, content="updated comment")

Comment updated successfully!
